In [1]:
import pandas as pd
from functions import *

# Фракционный счёт по данным WoS

В данном документе проиллюстрированы механики, которые позволяют посчитать один из показателей программы «Приоритет
2030», а именно:

> Количество публикаций в научных изданиях I и II квартилей, а также научных изданиях, включенных в индексы Arts and
> Humanities Citation Index (A&HCI) и Book Citation Index - Social Sciences & Humanities (BKCI-SSH), индексируемых в
> базе данных Web of Science Core Collection, в расчете на одного научно-педагогического работника (далее — НПР)

Исходный приказ *(актуален на момент создание данного документа 11.07.2021)*, в котором фигурирует показатель можно
найти по [ссылке](http://publication.pravo.gov.ru/Document/View/0001202106230024), в т.ч. в документе описана методика
расчёта показателя.

## Уточнение задачи

1. Для примера возьмём один из вузов — [Университет ИТМО](https://itmo.ru/) (почему именно этот университет я упомяну ниже в блоке про IT конференции)
1. Ограничимся одним годом для анализа — 2019
1. Используем относительно легко доступные данные — интерфейсы и выгрузки WoS доступные университетам по нац. подписке

## Аккаунты университетов в WoS

**Обратите внимание!** На момент написания WoS обновляло свои интерфейсы и последующие инструкции могут не сработать в
полном объёме через какое-то время.

1. Переходим к [интерфейсу поиска](https://apps.webofknowledge.com/WOS_GeneralSearch_input.do?product=WOS&search_mode=GeneralSearch&SID=E2IlyQDP2bZHZP9dgMJ&preferencesSaved=)
1. Далее переключаемся на [«Расширенный поиск»](https://apps.webofknowledge.com/WOS_AdvancedSearch_input.do?SID=E2IlyQDP2bZHZP9dgMJ&product=WOS&search_mode=AdvancedSearch)
1. В блоке обозначение полей ищем «OG= Профили организаций» и переходим по ссылке «[Указатель]».
1. В блоке поиска вбиваем какую-то часть официального названия университета на английском языке и наживаем на «Find»
(например для ИТМО хватило «ITMO»).
1. В списке найденного находим нашу организацию и нажав на «D» в гругляшке (старый UI WoS конечно «шикарен» 🤦‍),
переходим к расширенному списку названий организации.
1. Нажимаем на «Add» рядом с «Organization Name», что добавит основное название организации UI элемент, где написано
«Transfer your selected organization(s) below to the Organizations - Enhanced field on the search page.» и больше здесь,
пока, **ничего не нажимайте**.
1. Далее формируем исчерпывающий список названий университета для фракционного счёта, чтобы не возвращаться больше на
данную страницу. Я сохранил в файл `2021_07_07_wos_q1_q2_with_fractions/data/itmo_names.csv`.
1. В завершении нажимаем «Ok» внизу страницы и переходим к поиску публикаций по организации.

Теперь можно сформировать список названий и проверять произвольные строки на вхождение в конечный список названий:

In [2]:
itmo_names = pd.read_csv('data/itmo-names.csv', header=None, squeeze=True)
print(('ITMO' in itmo_names.unique()), ('Not ITMO' in itmo_names.unique()))

True False


---

Тем, кто знаком с консолью DevTools браузера (например в Chrome открывается по `F12`) упростит формирование списка
названий из **пункта № 7**, если вы введёте и запустите ниже представленный код:

```js
$('#SAOuterTable > tbody > tr:nth-child(1) > td > table > tbody > tr > td > table:nth-child(2) > tbody > tr > td > table > tbody > tr:nth-child(2) > td.SAContent > table tr').each(function() {
    console.log($(this).find('td:last > span').html())
})
```

В консоль выведется список, который можно скопировать в любой текстовый редактор и довольно быстро почистить с помощью
функции «Найти и заменить». Останется только вручную взять варианты названия из «Other Names».

---

## Работа с публикациями

Если в предыдущем блоке +/- всё было сделано правильно Вас должно перебросить на страницу расширенного поиска, с
указанным профилем организации (в моём случае `OG=(ITMO University)`).

1. Нажимаем кнопку «Поиск».
1. Ниже на странице в «Истории поиска» появится подборка с каким-то количеством результатов (в моём случае было найдено
15 563 документа).
1. Далее **Министерство придумало для нас квест**, который вынуждает нас не просто выгрузить выдачу полученную в предыдущем
пункте, а формировать несколько выборок с помощью фильтров и выгружать их.

Прежде чем мы перейдём к описанию решения квеста, опишем общие правила выгрузок, которые мы будем воспроизводить каждый
раз после решения части загадок (формирования удовлетворяющих выборок):

1. Нажимаем кнопку «Экспорт»
1. В выпадающем меню выбираем «Другие форматы файлов»
1. В сплывающем окне вводим следующие данные:
    * Записи с *такой-то* по *такую-то* (максимально за раз можно выгрузить по 500 записей, возможно Вам придётся
    сделать это несколько раз, что бы сформировать весь объём Ваших публикаций)
    * «Содержимое записи» устанавливаем в «Полная запись»
    * «Формат файла» выбираем «С разделителями табуляции (Win, UTF-8)»
1. Нажимаем на кнопку «Экспорт» и сохраняем файл в папку `2021_07_07_wos_q1_q2_with_fractions/data` данного проекта
(чтобы было понятней, рекомендую сохранять файлы по следующему шаблону `[publication_type]-[year]-[start]-[end].csv`,
например для научных статей для 2020 года название файла может выглядеть так:
`articles-and-reviews-2020-1-500.csv`)

### Статьи и обзоры за 2019 год

Работаем с блоком «Уточнение результатов» в нашей исходной выдаче по организации:

1. Нажимаем «Просмотреть все параметры» внизу блока «Уточнение результатов».
1. В блоке «Web of Science Index» нажимаем «дополнительные параметры / значения...».
1. В открывшемся интерфейсе проставляем галочки рядом с:
    * Science Citation Index Expanded
    * Social Sciences Citation Index
    * Arts and Humanities Citation Index
1. Нажимаем кнопку «Уточнить». Вас перебросит обратно сузив результаты выдачи.
1. Ставим галочки напротив интересующего нас года (2019) и нажимаем кнопку «Уточнить»
1. В блоке «Типы документов» ставим галочки напротив «ARTICLE» и «REVIEW», и наживаем на кнопку «Уточнить»

**Обратите внимание!** В процессе сужения результатов поиска по Вашему университету может отличаться
количество/вариативность сущностей с описанным выше. Это связано с тем, что система даёт работать, после каждого шага,
с сущностями, в которых есть статьи с аффилиацией Вашего университета.

**Ура!** Мы сформировали первую выборку для работы 😊

**Есть альтернативный вариант**, который мне более приятен, т.к. меньше переходов по интерфейсу и нажатий на кнопочки.
Но было важно показать все варианты использования интерфейса WoS.

1. Переходим на страницу [расширенного поиска](https://apps.webofknowledge.com/WOS_AdvancedSearch_input.do?SID=E2IlyQDP2bZHZP9dgMJ&product=WOS&search_mode=AdvancedSearch).
1. Вставляем в строку поиска следующий запрос:
```
OG=(ITMO University)
```
1. В блоке «Ограничить результаты по языкам и типам документов» с зажатым `Ctrl` отмечаем «Article» и «Review»,
дискриминацию по языкам не устраиваем.
1. В блоке «Период» выбираем «Настраиваемый диапазон лет» и с 2019 по 2019.
1. Выведем дополнительные настройки нажав на «ДУРУГИЕ ПАРАМЕТРЫ»
1. Web of Science Core Collection: указатели цитирования оставляем следующие галочки:
    * Science Citation Index Expanded (SCI-EXPANDED) --1975-по настоящее время
    * Social Sciences Citation Index (SSCI) --1975-по настоящее время
    * Arts & Humanities Citation Index (A&HCI) --1975-по настоящее время
1. Нажимаем на кнопку «Поиск» и переходим к результатам поиска.

У [выбранного мной университета](http://itmo.ru/) в конечном счёте осталось 1 025 публикаций удовлетворяющих
описанным выше ограничениям и фильтрам. Результаты экспортированы в файлы:

* `2021_07_07_wos_q1_q2_with_fractions/data/articles-and-reviews-2019-2018-1-500.csv`
* `2021_07_07_wos_q1_q2_with_fractions/data/articles-and-reviews-2019-2018-501-1000.csv`
* `2021_07_07_wos_q1_q2_with_fractions/data/articles-and-reviews-2019-2018-1001-1025.csv`

Соберём это в один dataframe и посмотрим на первые 5 строк:

In [3]:
ar = pd.concat([
    pd.read_csv('data/articles-and-reviews-2019-1-500.csv', sep='\t', index_col=False),
    pd.read_csv('data/articles-and-reviews-2019-501-1000.csv', sep='\t', index_col=False),
    pd.read_csv('data/articles-and-reviews-2019-1001-1025.csv', sep='\t', index_col=False),
]).reset_index(drop=True)
ar.head()

,PT,AU,BA,BE,GP,AF,BF,CA,TI,SO,...,PG,WC,SC,GA,UT,PM,OA,HC,HP,DA
0,J,"Tenishchev, SS; Kiselev, AD; Ivanov, AV; Uzdin...",NaN,NaN,NaN,"Tenishchev, Semen S.; Kiselev, Alexei D.; Ivan...",NaN,NaN,Multiple minimum-energy paths and scenarios of...,PHYSICAL REVIEW E,...,11,"Physics, Fluids & Plasmas; Physics, Mathematical",Physics,JZ3EB,WOS:000504984100002,31962453.0,NaN,NaN,NaN,2021-07-20
1,J,"Bai, M; Huang, H; Liu, Z; Zhan, TT; Xia, SF; L...",NaN,NaN,NaN,"Bai, Min; Huang, Hui; Liu, Zhe; Zhan, Tingting...",NaN,NaN,InAs/InP core/shell nanowire gas sensor: Effec...,APPLIED SURFACE SCIENCE,...,7,"Chemistry, Physical; Materials Science, Coatin...",Chemistry; Materials Science; Physics,JA4ZH,WOS:000487851700017,NaN,NaN,NaN,NaN,2021-07-20
2,J,"Mamontov, OV; Kozlenok, AV; Kamshilin, MA; Shl...",NaN,NaN,NaN,"Mamontov, Oleg, V; Kozlenok, Andrey, V; Kamshi...",NaN,NaN,The Autonomic Regulation of Circulation and Ad...,CARDIOLOGY RESEARCH AND PRACTICE,...,6,Cardiac & Cardiovascular Systems,Cardiovascular System & Cardiology,KC6FG,WOS:000507270500001,32082622.0,"DOAJ Gold, Green Published",NaN,NaN,2021-07-20
3,J,"Pozina, G; Hemmingsson, C; Belonovskii, AV; Le...",NaN,NaN,NaN,"Pozina, Galia; Hemmingsson, Carl; Belonovskii,...",NaN,NaN,Emission Properties of GaN Planar Hexagonal Mi...,PHYSICA STATUS SOLIDI A-APPLICATIONS AND MATER...,...,5,"Materials Science, Multidisciplinary; Physics,...",Materials Science; Physics,MV8SO,WOS:000504787900001,NaN,NaN,NaN,NaN,2021-07-20
4,J,"Smirnov, AM; Young, EC; Bougrov, VE; Speck, JS...",NaN,NaN,NaN,"Smirnov, A. M.; Young, E. C.; Bougrov, V. E.; ...",NaN,NaN,Stress relaxation in semipolar and nonpolar II...,JOURNAL OF APPLIED PHYSICS,...,11,"Physics, Applied",Physics,KA1CY,WOS:000505538100009,NaN,NaN,NaN,NaN,2021-07-20


### Книжки за 2019 год

1. Переходим на страницу [расширенного поиска](https://apps.webofknowledge.com/WOS_AdvancedSearch_input.do?SID=E2IlyQDP2bZHZP9dgMJ&product=WOS&search_mode=AdvancedSearch).
1. Вставляем в строку поиска следующий запрос:
```
OG=(ITMO University)
```
1. В блоке «Ограничить результаты по языкам и типам документов» с зажатым `Ctrl` отмечаем «Book»,
дискриминацию по языкам не устраиваем.
1. В блоке «Период» выбираем «Настраиваемый диапазон лет» и с 2019 по 2019.
1. Web of Science Core Collection: указатели цитирования оставляем следующие галочки:
    * Book Citation Index– Social Sciences & Humanities (BKCI-SSH) --2005-по настоящее время
1. Нажимаем на кнопку «Поиск» и переходим к результатам поиска.

К сожалению у [ИТМО](http://itmo.ru/) таких документов не оказалось, экспортировать не с чем 🤷 **Но даже больше!** Я не
смог найти такого рода документов по всей стране за 2018-2020 года, вместо `OG=(ITMO University)` вбивал `CU=(RUSSIA)`,
т.е осуществлял поиск по всей стране, но результат аналогичный. Видимо что-то на перспективу заложено 🤔

### Бонусная IT-выборка

Данная выборка далась тяжело, возможно мне не повезло, но пришлось перебрать 7-10 университетов, прежде чем в выдаче
появились публикации, на которых можно было проиллюстрировать механику подсчёта показателя (даже вузы из
[5-100](https://www.5top100.ru/) не подходили).

1. Переходим на страницу [расширенного поиска](https://apps.webofknowledge.com/WOS_AdvancedSearch_input.do?SID=E2IlyQDP2bZHZP9dgMJ&product=WOS&search_mode=AdvancedSearch).
1. Вставляем в строку поиска следующий запрос:
```
OG=(ITMO University) AND
WC=(COMPUTER SCIENCE INFORMATION SYSTEMS OR COMPUTER SCIENCE ARTIFICIAL INTELLIGENCE OR COMPUTER SCIENCE THEORY METHODS OR COMPUTER SCIENCE INTERDISCIPLINARY APPLICATIONS OR COMPUTER SCIENCE SOFTWARE ENGINEERING OR COMPUTER SCIENCE CYBERNETICS OR COMPUTER SCIENCE HARDWARE ARCHITECTURE)
```
1. В блоке «Ограничить результаты по языкам и типам документов» с зажатым `Ctrl` отмечаем «Proceedings Paper»,
дискриминацию по языкам не устраиваем.
1. В блоке «Период» выбираем «Настраиваемый диапазон лет» и с 2018 по 2020.
1. Web of Science Core Collection: указатели цитирования оставляем следующие галочки:
    * Conference Proceedings Citation Index- Science (CPCI-S) --1990-по настоящее время
1. Нажимаем на кнопку «Поиск» и переходим к результатам поиска.

У меня найдено 234 документа, результаты экспортированы в файлы:

* `2021_07_07_wos_q1_q2_with_fractions/data/it-conf-2019-1-234.csv`

**Отлично!** Сформирован комплект публикаций в конференциях. Давайте посмотрим на пример данных.

In [4]:
it = pd.read_csv('data/it-conf-2019-1-234.csv', sep='\t', index_col=False).reset_index(drop=True)
it.head()

,PT,AU,BA,BE,GP,AF,BF,CA,TI,SO,...,PG,WC,SC,GA,UT,PM,OA,HC,HP,DA
0,J,"Khartov, AA; Zani, M",NaN,NaN,NaN,"Khartov, A. A.; Zani, M.",NaN,NaN,Asymptotic analysis of average case approximat...,JOURNAL OF COMPLEXITY,...,21,"Computer Science, Theory & Methods; Mathematic...",Computer Science; Mathematics,HT3UY,WOS:000464489900003,NaN,NaN,NaN,NaN,2021-07-20
1,J,"Kovalchuk, SV; Krzhizhanovskaya, VV; Shi, Y; F...",NaN,NaN,NaN,"Kovalchuk, Sergey V.; Krzhizhanovskaya, Valeri...",NaN,NaN,"Science at the intersection of data, modelling...",JOURNAL OF COMPUTATIONAL SCIENCE,...,3,"Computer Science, Interdisciplinary Applicatio...",Computer Science,IM3TH,WOS:000477917200009,NaN,NaN,NaN,NaN,2021-07-20
2,J,"Bates, S; Leonenko, V; Rineer, J; Bobashev, G",NaN,NaN,NaN,"Bates, Savannah; Leonenko, Vasiliy; Rineer, Ja...",NaN,NaN,Using synthetic populations to understand geos...,COMPUTATIONAL AND MATHEMATICAL ORGANIZATION TH...,...,12,"Computer Science, Interdisciplinary Applicatio...",Computer Science; Mathematics; Mathematical Me...,HQ2BV,WOS:000462205200004,NaN,NaN,NaN,NaN,2021-07-20
3,J,"Bocharova, IE; Kudryashov, BD; Skachek, V; Yak...",NaN,NaN,NaN,"Bocharova, Irina E.; Kudryashov, Boris D.; Ska...",NaN,NaN,BP-LED Decoding Algorithm for LDPC Codes Over ...,IEEE TRANSACTIONS ON INFORMATION THEORY,...,17,"Computer Science, Information Systems; Enginee...",Computer Science; Engineering,HM5BO,WOS:000459489600023,NaN,NaN,NaN,NaN,2021-07-20
4,B,"Akhtiamov, O; Siegert, I; Karpov, A; Minker, W",NaN,"Nakamura, S; Gasic, M; Zuckerman, I; Skantze, ...",NaN,"Akhtiamov, Oleg; Siegert, Ingo; Karpov, Alexey...",NaN,NaN,Cross-Corpus Data Augmentation for Acoustic Ad...,20TH ANNUAL MEETING OF THE SPECIAL INTEREST GR...,...,10,"Computer Science, Interdisciplinary Applicatio...",Computer Science; Linguistics,BQ4NT,WOS:000591510500033,NaN,NaN,NaN,NaN,2021-07-20


Как мы видим по 2-м примерам название колонок сходу ничего не говорит, путём визуального анализа фиксируем следующее:

* **AU** — авторы
* **C1** — список аффилиаций с привязанными к ним авторами
* **SN** — ISSN
* **EI** — eISSN
* **DI** — DOI
* **CT** — название конференции

На всё остальное в данной аналитике можно забить, хватит вышеупомянутых столбцов.

### Квартильность публикаций

В исходном квесте от министерства есть ограничения, что статьи должны быть I и II квартиля. Фильтр по квартилю к книгам
и публикациям в конференциях я не применял, мне показалось это логичным (аргументированно поспорить можно
[здесь](https://t.me/psalgroup) или [здесь](https://github.com/psalru/analytics/issues))

Нам потребуется таблица, которая будет содержать ISSN и eISSN журналов, а также их квартиль. Данные можно взять с
[данной страницы](https://jcr.clarivate.com/jcr/browse-journals), предварительно применив следующие фильтры:

* В «Citation Indexes» оставляем только интересующие нас индексы:
    * Science Citation Index Expanded (SCIE)
    * Social Science Citation Index (SSCI)
    * Arts & Humanities Citation Index (AHCI)
* В «JIF Quartile» ставим галочки напротив «Quartile 1» и «Quartile 2»
* «JCR Year» последовательно выставляем по годам: 2019

У меня получилось 6 372 журналов за 2019 год. Если вы не знакомы с DevTools браузера придётся копировать данные в
ручную, но если таковые знания есть обратите внимание на вкладку «Network» и тип прилетающих данных со страницы «XHR».
Это существенно ускорит процесс формирования необходимых данных для аналитики 😏

У меня получилось что-то вроде такого (выведены первые 5 записей):

In [5]:
wos_quartiles = pd.read_csv('data/wos-journals-q1-q2.csv', sep=',').fillna('****-****')
wos_quartiles.head()

,jcrYear,issn,eissn,quartile
0,2019,0007-9235,1542-4863,Q1
1,2019,0028-4793,1533-4406,Q1
2,2019,2058-8437,2058-8437,Q1
3,2019,1474-1776,1474-1784,Q1
4,2019,0140-6736,1474-547X,Q1


## Фракционный счёт

Будем считать методом фракционного счёта полученные данные:

1. «Articles» and «Reviews» из индексов SCI-EXPANDED, SSCI и A&HCI
2. По «Books» из индекса BKCI-SSH по [ИТМО](http://itmo.ru/) считать нечего 🤷‍
3. IT-конференции из индекса CPCI-S

Для начало добавим столбец `authors_count` с общим количеством авторов.

In [6]:
ar['authors_count'] = ar['AU'].apply(lambda x: len(x.split('; ')))
ar[['DI', 'authors_count']].head()

,DI,authors_count
0,10.1103/PhysRevE.100.062704,4
1,10.1016/j.apsusc.2019.143756,10
2,10.1155/2019/8391924,4
3,10.1002/pssa.201900894,11
4,10.1063/1.5126195,5


In [7]:
it['authors_count'] = it['AU'].apply(lambda x: len(x.split('; ')))
it[['DI', 'authors_count']].head()

,DI,authors_count
0,10.1016/j.jco.2018.04.001,2
1,10.1016/j.jocs.2019.05.005,7
2,10.1007/s10588-018-09281-2,4
3,10.1109/TIT.2018.2866879,4
4,NaN,4


Все ниже используемые вспомогательные функции можно изучить
[по ссылке](https://github.com/psalru/analytics/blob/master/2021_07_07_wos_q1_q2_with_fractions/functions.py), а тесты к ним вы увидите
[здесь](https://github.com/psalru/analytics/blob/master/2021_07_07_wos_q1_q2_with_fractions/tests.py)

Следующим шагом посчитаем общее количество аффилиаций в каждой публикации и запишем их в `affiliation_count` и
сформируем данные, которые нам нужны для фракционного счёта `my_affiliation_data`. **Важно**, что при подсчёте
аффилиаций не считаются дубли, а по исследуемому университету происходят объединения даже по разным написаниям
организации.

In [8]:
ar['affiliation_count'] = ar['C1'].apply(lambda x: get_affiliation_count(x, itmo_names.unique()))
ar['my_affiliation_data'] = ar['C1'].apply(lambda x: get_my_affiliation_data(x, itmo_names.unique()))
ar[['DI', 'authors_count', 'affiliation_count']].head()

,DI,authors_count,affiliation_count
0,10.1103/PhysRevE.100.062704,4,3
1,10.1016/j.apsusc.2019.143756,10,5
2,10.1155/2019/8391924,4,3
3,10.1002/pssa.201900894,11,4
4,10.1063/1.5126195,5,2


In [9]:
it['affiliation_count'] = it['C1'].apply(lambda x: get_affiliation_count(x, itmo_names.unique()))
it['my_affiliation_data'] = it['C1'].apply(lambda x: get_my_affiliation_data(x, itmo_names.unique()))
it[['DI', 'authors_count', 'affiliation_count', 'my_affiliation_data']].head()

,DI,authors_count,affiliation_count,my_affiliation_data
0,10.1016/j.jco.2018.04.001,2,3,"{'packages': [{'name': 'ITMO Univ', 'authors':..."
1,10.1016/j.jocs.2019.05.005,7,7,"{'packages': [{'name': 'ITMO Univ', 'authors':..."
2,10.1007/s10588-018-09281-2,4,3,"{'packages': [{'name': 'ITMO Univ', 'authors':..."
3,10.1109/TIT.2018.2866879,4,2,{'packages': [{'name': 'St Petersburg Natl Res...
4,NaN,4,4,"{'packages': [{'name': 'ITMO Univ', 'authors':..."


Теперь можно посчитать фракционность статей и записать их в `fraction`.

In [10]:
ar['fraction'] = ar.apply(lambda x: get_faction(x), axis=1)
ar[['DI', 'authors_count', 'affiliation_count', 'my_affiliation_data', 'fraction']].head()

,DI,authors_count,affiliation_count,my_affiliation_data,fraction
0,10.1103/PhysRevE.100.062704,4,3,"{'packages': [{'name': 'ITMO Univ', 'authors':...",0.375000
1,10.1016/j.apsusc.2019.143756,10,5,"{'packages': [{'name': 'ITMO Univ', 'authors':...",0.150000
2,10.1155/2019/8391924,4,3,"{'packages': [{'name': 'ITMO Univ', 'authors':...",0.250000
3,10.1002/pssa.201900894,11,4,"{'packages': [{'name': 'ITMO Univ', 'authors':...",0.287879
4,10.1063/1.5126195,5,2,"{'packages': [{'name': 'ITMO Univ', 'authors':...",0.600000


In [11]:
it['fraction'] = it.apply(lambda x: get_faction(x), axis=1)
it[['DI', 'authors_count', 'affiliation_count', 'my_affiliation_data', 'fraction']].head()

,DI,authors_count,affiliation_count,my_affiliation_data,fraction
0,10.1016/j.jco.2018.04.001,2,3,"{'packages': [{'name': 'ITMO Univ', 'authors':...",0.250000
1,10.1016/j.jocs.2019.05.005,7,7,"{'packages': [{'name': 'ITMO Univ', 'authors':...",0.261905
2,10.1007/s10588-018-09281-2,4,3,"{'packages': [{'name': 'ITMO Univ', 'authors':...",0.250000
3,10.1109/TIT.2018.2866879,4,2,{'packages': [{'name': 'St Petersburg Natl Res...,0.500000
4,NaN,4,4,"{'packages': [{'name': 'ITMO Univ', 'authors':...",0.125000


## Фильтрация данных

Фракции посчитаны, но нам ещё нужно отсечь по обоим массивам публикации, которые полностью не удовлетворяют методическим
требования, а именно:

1. IT-шные Proceedings Paper должны быть из эксклюзивного списка;
1. Articles & Reviews должны быть I и II квартиля;

Для решения задачи с конференциями я сформировал файл `conferences.csv` с перечислением всех аббревиатур конференций.
Давайте попробуем проставить признак `is_good` для конференций на базе данных и этого файла.

In [12]:
conferences = pd.read_csv('data/conferences.csv', header=None, squeeze=True)

it['is_good'] = it['CT'].apply(lambda x: is_good_conf(x, conferences.unique()))
it[it['is_good']][['DI', 'authors_count', 'affiliation_count', 'fraction']].head()

,DI,authors_count,affiliation_count,fraction
42,10.1109/ICCVW.2019.00068,8,2,0.25
61,10.1145/3299904.3340317,3,2,0.50


На выходе получилось только 2 Proceedings Paper, которые удовлетворяют условиям.

Перейдём теперь к проставлению `is_good` в Articles & Reviews. Для этого для начала затащим в данные квартили.

In [13]:
issn = ar.merge(wos_quartiles, left_on='SN', right_on='issn', how='left')
eissn = ar.merge(wos_quartiles, left_on='EI', right_on='eissn', how='left')
issn.update(eissn)
ar = issn
ar['is_good'] = ar['quartile'].apply(lambda x: (x == 'Q1') | (x == 'Q2'))
ar[['DI', 'authors_count', 'affiliation_count', 'fraction', 'quartile', 'is_good']].head(10)

,DI,authors_count,affiliation_count,fraction,quartile,is_good
0,10.1103/PhysRevE.100.062704,4,3,0.375000,Q1,True
1,10.1016/j.apsusc.2019.143756,10,5,0.150000,Q1,True
2,10.1155/2019/8391924,4,3,0.250000,NaN,False
3,10.1002/pssa.201900894,11,4,0.287879,NaN,False
4,10.1063/1.5126195,5,2,0.600000,Q2,True
5,10.1021/acsomega.9b03036,8,4,0.125000,Q2,True
6,10.1103/PhysRevB.100.235444,5,5,0.566667,Q2,True
7,10.1063/1.5129207,15,11,0.100000,Q2,True
8,10.1080/1536383X.2019.1697684,7,3,0.142857,NaN,False
9,10.1002/rnc.4861,4,6,0.125000,Q1,True


В целом мы справились и можно выполнить окончательные расчёты.

In [14]:
pd.Series({
    'ar_count': len(ar),
    'ar_good': len(ar[ar['is_good']]),
    'ar_fraction': ar[ar['is_good']]['fraction'].sum(),
    'it_count': len(it),
    'it_good': len(it[it['is_good']]),
    'it_fraction': it[it['is_good']]['fraction'].sum(),
    'fraction_total': ar[ar['is_good']]['fraction'].sum() + it[it['is_good']]['fraction'].sum()
})

ar_count          1025.000000
ar_good            667.000000
ar_fraction        239.091955
it_count           234.000000
it_good              2.000000
it_fraction          0.750000
fraction_total     239.841955
dtype: float64

Показатель для ИТМО за 2019 год `fraction_total` и составляет **239.841955**.

Если у Вас есть вопросы, а ещё лучше улучшения к данному материалу приглашаю Вас к открытому обсуждению в
Телеграм [группе](https://t.me/psalgroup) посвящённой «Приоритету 2030» или ещё лучше, **присылайте свои issues-ы**
сразу [в репозиторий](https://github.com/psalru/analytics/issues).